In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
import scipy
import sys
import imp

Using TensorFlow backend.


In [2]:
from layers import AttentionLayer
from data_util import unpack_news_data,data_generator
from process_data import load_data,get_rank_of_size

In [3]:
rank = get_rank_of_size()
rank.remove('XOM')
rank.remove('GOOG')
codes = rank[21:31]

In [13]:
rank = get_rank_of_size()
rank.remove('XOM')
rank.remove('GOOG')
print(rank[21:31])

['PFE', 'CMCSA', 'JNJ', 'MCD', 'MRK', 'HD', 'AABA', 'CSCO', 'DAL', 'V']


In [4]:
prices_dir = 'data2/prices/'
pdata = dict()
for code in codes:
    pdata[code] = pd.read_csv(prices_dir+'stockPrices_%s.csv'%(code))
    pdata[code]['Date'] = pd.to_datetime(pdata[code]['Date'])

In [20]:
model =  load_model('model/h_10.h5', custom_objects = {'AttentionLayer': AttentionLayer})

In [6]:
data = load_data(codes,encoder_kind='Bert',with_date=True)

data of PFE is loaded
data of CMCSA is loaded
data of JNJ is loaded
data of MCD is loaded
data of MRK is loaded
data of HD is loaded
data of AABA is loaded
data of CSCO is loaded
data of DAL is loaded
data of V is loaded


In [7]:
def get_pred(code):
    pre = model.predict(x=[unpack_news_data(data[code]['train'][0]),data[code]['train'][1]],
                               verbose=0,batch_size=64)
    y_pred = np.argmax(pre,axis=-1)
    y_true = np.argmax(data[code]['train'][2],axis=-1)
    # y_match = y_true == y_pred
    # y_match = result.astype(np.int)
    return y_pred

In [8]:
sim = dict()
for code in codes:
    sim[code] = list()
    y_pred = get_pred(code)
    for i in range(len(y_pred)):
        pred = y_pred[i]
        date = data[code]['train'][3][i]
        sim[code].append((date,pred))

In [25]:
def simulate(pred_data,prices_data,days=1):
    earnings = 0
    batch = 10000
    for i in range(len(pred_data)):
        dt = pred_data[i][0]
        pr = pred_data[i][1]
        prices = prices_data[prices_data['Date']>=dt][:days]
        high_price = max(prices['High'].values)
        low_price = max(prices['Low'].values)
        open_price = prices['Open'].values[0]
        close_price = prices['Close'].values[-1]
        if pr == 1:
            change_rate = close_price/open_price-1
            day_rate = high_price/open_price-1
            if day_rate >= 0.02:
                change_rate = 0.02
        else:
            change_rate = 1-close_price/open_price
            day_rate = 1-low_price/open_price
            if day_rate >= 0.01:
                change_rate = 0.01
        earnings += batch*change_rate
    return earnings

In [32]:
earnings = dict()
for code in codes:
    earnings[code]=simulate(sim[code],pdata[code],1)/10000

In [33]:
earnings

{'AABA': -0.23767957862122116,
 'CMCSA': 0.3463553246206639,
 'CSCO': 0.5512638761600368,
 'DAL': 0.34938036850047116,
 'HD': 0.660202353810423,
 'JNJ': 0.09929041644848577,
 'MCD': 0.2170953669195333,
 'MRK': 0.0803821234314731,
 'PFE': 0.18537438072312762,
 'V': 0.3118810795847373}